In [ ]:
#length of each comment category.
#https://www.kaggle.com/code/abhinavjhanwar/word-embedding-keras-lstm-classification
#model has less threat comments, so it predicting less threats.

### Methods :
    
1) LSTM Sequential Model (TextVectorization)

2) Logistic Regression (TFIDF)

3) NB-SVM strong linear baseline



# Importing essential libraries 

In [99]:
#General Libraries for Data Pre-Processing and Analysis.
import pandas as pd
import numpy as np
import re
import scipy
import math
from collections import Counter
import gensim
import string
from pandas_profiling import ProfileReport

#Libraries for visualisation.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
!pip install gradio
import gradio as gr


#Libraries for NLP (for data pre-processing.)
import nltk
    
#Libraries for feature engineering,model bulding,evalution.

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import metrics
#from sklearn.metrics import cross_val_score
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text
from keras.utils import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint

#To disable warnings.
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Exploration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing datasets in python.
train = pd.read_csv("/content/drive/MyDrive/dataset/train.csv")
test = pd.read_csv('/content/drive/MyDrive/dataset/test.csv')
test_y = pd.read_csv("/content/drive/MyDrive/dataset/test_labels.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/dataset/sample_submission.csv")

#### dataset info

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            153164 non-null  object
 1   comment_text  153164 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


#### Missing values checks

In [ ]:
train.isnull().sum() #Looking for missing values in training dataset.

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [ ]:
test.isnull().sum() #Looking for missing values in testing dataset.

id              0
comment_text    0
dtype: int64

###### Data preparation for analysis.

In [ ]:
# train_profile = ProfileReport(train)
# train_profile.to_widgets()
# #train_profile

In [ ]:
# test_profile = ProfileReport(test)
# test_profile.to_widgets()

In [ ]:
labels = ['toxic','severe_toxic', 'obscene', 'identity_hate', 'insult', 'threat']
target = train[labels]

#Only training data comments.
train_data= train['comment_text']

#Testing data comments.
test_data = test['comment_text']

#All comments together (Train + Test)
data = pd.concat([train_data, test_data])

#Since we have are aiming for six types of comments for classification. We can later use this list as target_list for further analysis.
targets = list(train.columns[2:])

# #Length of each comment, storing it in a new column.
comment_length = train.comment_text.apply(lambda x: len(x))

#creating two dataframes for toxic and non-toxic.
toxic_1 = sum(train['toxic'] == 1)
toxic_0 = sum(train['toxic'] == 0)

obscene_1 = sum(train['obscene'] == 1)
obscene_0 = sum(train['obscene'] == 0)

insult_1 = sum(train['insult'] == 1)
insult_0 = sum(train['insult'] == 0)

severe_toxic_1 = sum(train['severe_toxic'] == 1)
severe_toxic_0 = sum(train['severe_toxic'] == 0)

identity_hate_1 = sum(train['identity_hate'] == 1)
identity_hate_0 = sum(train['identity_hate'] == 0)

threat_1 = sum(train['threat'] == 1)
threat_0 = sum(train['threat'] == 0)

bars1 = [toxic_1,severe_toxic_1,obscene_1,identity_hate_1,insult_1,threat_1]
bars2 = [toxic_0,severe_toxic_0,obscene_0,identity_hate_0,insult_0,threat_0]

label_0 = pd.DataFrame(list(zip(labels, bars2)), columns = ['label', 'counts'])
label_1 = pd.DataFrame(list(zip(labels, bars1)), columns = ['label', 'counts'])

#Since we have are aiming for six types of comments for classification. We can later use this list as target_list for further analysis.
targets = list(train.columns[2:])

#Fetching only classification data for calculation.
target_data = train[targets]

#Creating a variable to store toxic comments.
toxic = target_data.sum(axis=1)
toxic = (toxic > 0)

#Creating a lable all comment feature
targets.append('any_label')
target_data['any_label'] = toxic
target['any_label'] = toxic

#overlapping
target_sums = np.sum(target[target.columns[:-1]], axis=1)
target_sums = target_sums[(target.any_label==1)]

##### EDA of comments.

In [ ]:
#need to claen bit of.

def EDA(train):

    eda = {}  #storing analysis values in this dict.

    for label in targets:
        label_counts = []
        others = targets
        comment = target_data[(target_data[label]==1)]
        others.remove(label)
        label_counts.append(('total', len(comment)))
        for other in others:
                label_counts.append((other, comment[other].sum()))
        eda[label] = label_counts

    print('The Training dataset has, %d out of %d comments, or %.2f%%, toxic comments.' % 
     (np.sum(toxic), len(train), (np.sum(toxic)/len(train))*100))

    totals = []
    for key, value in eda.items():
        totals.append(value[0][1])
        print('\n')
        print('-'*30+key+'-'*30)
        print('\nThere are %d %s comments. (%.2f%% of all data.)' % (value[0][1], key, (value[0][1]/len(train))*100))

        for cnt in value[1:]:
            print('- From that, %d or %.2f%% were also %s.' % (cnt[1], (cnt[1]/value[0][1])*100, cnt[0]))
            
    print('\n')       
    print('-'*30+'Comment Type overlapping over each other.'+'-'*30)
    for i in range(1,len(target.columns)):
        print('%d label%s: %.1f%%' % (i, 's' if i > 1 else '', 100 * len([s for s in target_sums if s == i])/len(target_sums)))
    return eda,totals

In [ ]:
if __name__== "__main__":
    eda,totals = EDA(train)

The Training dataset has, 16225 out of 159571 comments, or 10.17%, toxic comments.


------------------------------toxic------------------------------

There are 15294 toxic comments. (9.58% of all data.)
- From that, 1595 or 10.43% were also severe_toxic.
- From that, 7926 or 51.82% were also obscene.
- From that, 449 or 2.94% were also threat.
- From that, 7344 or 48.02% were also insult.
- From that, 1302 or 8.51% were also identity_hate.
- From that, 15294 or 100.00% were also any_label.


------------------------------obscene------------------------------

There are 8449 obscene comments. (5.29% of all data.)
- From that, 1517 or 17.95% were also severe_toxic.
- From that, 301 or 3.56% were also threat.
- From that, 6155 or 72.85% were also insult.
- From that, 1032 or 12.21% were also identity_hate.
- From that, 8449 or 100.00% were also any_label.


------------------------------insult------------------------------

There are 7877 insult comments. (4.94% of all data.)
- From tha

#### Visulisations

###### - Pie Chart

In [ ]:
label_count = {}
for label in labels:
    label_count[label] = len(train[train[label] == 1])
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
label_count = {keys: values for keys, values in sorted(label_count.items(), key = lambda item: item[1], reverse = True)}
values=[*label_count.values()]

fig = px.pie(values=values,names=labels,hole=.3)

fig.update_traces(hoverinfo='label', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title_text="Comment Types(Labels) propotion in the dataset",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Labels', x=0.50, y=0.5, font_size=20, showarrow=False)])

fig.show()

In our train dataset only 10% of the data was toxic. Out of that 10%, 43.6% are toxic, 24.1% are severe_toxic,
4% of the toxic comments made are insults, 4.54% are identity attacks, 22.4% are obscene, 1.36% are threats.

###### Bar chart for each comment type counts 

In [ ]:
fig = px.bar(label_1, x='label', y='counts',color='counts', height=500)
fig.update_traces(text =label_1['counts'], texttemplate='%{text:.3s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(
    title="Number of comments for each comment types", # title of plot
    xaxis_title_text='Comment Types', # xaxis label
    yaxis_title_text='Number of Comments', # yaxis label
    bargap=0.01, # gap between bars of adjacent location coordinates
    bargroupgap=0.01) # gap between bars of the same location coordinates
fig.show()

##### Length Distribution 

In [ ]:
fig = px.histogram(x=train.comment_text.apply(lambda x: len(x)))

fig.update_layout(
    title="Length Distribution", # title of plot
    xaxis_title_text='Length of the Comments', # xaxis label
    yaxis_title_text='Numner of Comments', # yaxis label
    bargap=0.01, # gap between bars of adjacent location coordinates
    bargroupgap=0.01 # gap between bars of the same location coordinates
)
fig.show()

In [ ]:
for label in labels:
    fig = px.histogram(x=train[train[label]==1].comment_text.apply(lambda x: len(x)))
    fig.update_layout(
        title=label, 
        xaxis_title_text='Length of the Comments',
        yaxis_title_text='Numner of Comments',
        bargap=0.01, 
        bargroupgap=0.01 
    )
    fig.show()

##### Comment Types distribution

In [ ]:
plt.figure(figsize=(12,6))
for label in labels:
    sns.distplot(target[label],kde=True,hist=False,bins=240, label=label)
plt.title('Distribution of comments types in the training data.')
plt.legend()
plt.show()

###### Bar chart comparision for each comment type where comment is clean and toxic. 

In [ ]:
#cleaned.
# Code to draw bar graph for visualising distribution of classes within each label.

fig = go.Figure(data=[
    go.Bar(name='Labeled = 1', x=label_1['label'],y=label_1['counts'],text = label_1['counts']),
    go.Bar(name='Labeled = 0', x=label_0['label'],y=label_0['counts'],text = label_0['counts'])])
fig.update_layout(barmode='group')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(
    title="Number of comments where comment is toxic(labeled=1 and non-toxic(lable=0)", # title of plot
    xaxis_title_text='Comment Types', # xaxis label
    yaxis_title_text='Number of Comments', # yaxis label
    bargap=0.01, # gap between bars of adjacent location coordinates
    bargroupgap=0.01) # gap between bars of the same location coordinates
fig.show()

#####  Cross correlation matrix across labels

In [ ]:
fig = px.imshow(train[labels].corr(), text_auto=True,aspect = 'auto',
               labels=dict(x="Comment Types", y="Comment Types"),color_continuous_scale='RdBu_r')
fig.show()

As seen in the cross-correlation matrix, there is a high chance of obscene comments to be insulting.

In order to get an idea of what are the words that contribute the most to different labels, we write a function to generate word clouds. The function takes in a parameter label (i.e., toxic, insult, threat, etc)

##### Wordcloud for each comment type 

In [ ]:
#cleaned.

def word_cloud(text):
    comment_type = train[train[text]==1]
    comments = comment_type['comment_text']
    toxic_comment = pd.Series(comments).str.cat(sep = '')
    
     # Create and generate a word cloud image
    wordcloud = WordCloud(background_color="white", max_words=1000, 
                          contour_width=3, contour_color='firebrick',width=1500, height=700,
                          max_font_size=150).generate(toxic_comment)
    plt.figure(figsize=[15,10])
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Most common words in {text} comment", size=25)
    plt.show()

In [ ]:
if __name__== "__main__":
    for head in labels:
        word_cloud(head)

## Data Pre-Processing

Before fitting models, we need to break down the sentence into unique words by tokenizing the comments. In the tokenize() function, we remove punctuations and special characters. We also filtered out non-ascii characters after observing the results of feature engineering. We then lemmatize the comments and filter out comments with length below 3. Besides lemmatization, we also tried stemming but did not get a better result.

In [ ]:
def lower_text(text):
    return text.lower()

In [ ]:
def removing_url(text):
  text = re.sub(r"\b(?:(?:https|ftp|http|www)://)?\w[\w-]*(?:\.[\w-]+)+\S*", '', text, flags=re.MULTILINE)
  return text

In [ ]:
def remove_punctuation(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    text = regex.sub(" ", text)
    return text

In [ ]:
def remove_common_words(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
def remove_html_tags(text):
    html = re.compile(r'<.*?>')
    text = html.sub(r'',text)
    return text

In [ ]:
# remove leaky elements like ip,user
def remove_ips(text):
    text=re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",text)
    return text

In [ ]:
def clean_comments(text):
  text = lower_text(text)
  text = removing_url(text)
  text = remove_punctuation(text)
  text = remove_common_words(text)
  text = remove_ips(text)
  #text = remove_stop_words(text)
  #text = lemmatize(text)
  return text

In [ ]:
# # a = df['comment_text'][1]
# # b= clean_comments(df['comment_text'][1])
# print(a)
# print(b)

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda x :clean_comments(x))

## Feature Engineering

##### Preparing data for different models using different feature engineering methods.

Benchmarking Different Vectorizer


We determined to use TFIDF to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus.

Besides TFIDF, we also tried CountVectorizer. However, it is not performing as well as TFIDF. The TfidfVectorizer is actually CountVectorizer followed by TfidfTransformer. TfidfTransformer transforms a count matrix to a normalized Term-Frequency or TermFrequency-InverseDocumentFrequency representation. The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus. That's why we can improve the accuracy here.

For example: Since this corpus consists of data from the Wikipedia's talk page edits, the words such as wiki, Wikipedia, edit, page are very common. But for our classification purposes they do not provide us useful information and that should probably be the reason why TFIDF worked better than CountVectorizer.

# Feature Engineering

In [ ]:
#provide all text vocab to lstm and tfid

In [ ]:
train_x = train['comment_text'].values
train_y = train[labels].values
train_comment = train['comment_text']
test_comment = test['comment_text']
all_comments = pd.concat([train_comment, test_comment])

### Naive Bayes & Logistric Regression

In [ ]:
def tfid(data):
    
    #creating seperate vectorizer for characters and words.
    
    word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,3),
    max_features=50000,
    stop_words = 'english',
    dtype=np.float32)
    
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    token_pattern=r'\w{1,}',
    ngram_range=(3,6),
    max_features=50000,
    stop_words = 'english',
    dtype=np.float32
    )
    
    #feeding the words and characters into vocab.
    char_vectorizer.fit(data)
    word_vectorizer.fit(data)
    
    #transforming the comments ngram of characters and words.
    char = char_vectorizer.transform(data)
    word = word_vectorizer.transform(data)
    
    data = hstack([char,word])
    
    return data

In [ ]:
if __name__== "__main__":
    train_x = tfid(train_comment)

In [ ]:
if __name__== "__main__":
    test_x = tfid(test_comment)

###  LSTM Baseline model with Tokenization 
The Tokenizer, as the name suggests, tokenizes the text. Tokenization is the process of splitting text to individual elements (character, word, sentence, etc).

In [ ]:
def tokenize_padding(data):
    
    #Tokenizing
    word_tokenizer = Tokenizer(num_words=100000)
    word_tokenizer.fit_on_texts(list(data))
    data = tokenizer.texts_to_sequences(data)
    
    #Padding
    data = keras.utils.pad_sequences(data, maxlen=100)
    
    return data

In [ ]:
if __name__== "__main__":
    lstm_train_x = tokenize_padding(train['comment_text'].values)

In [ ]:
if __name__== "__main__":
    lstm_test_x = tokenize_padding(test['comment_text'].values)

### LSTM Bidirectional with TextVectorization 

- TextVectorization converts the text to integer sequences.
- The vocabulary for the layer must be either supplied on construction or learned via adapt(). When this layer is adapted, it will analyze the dataset, determine the frequency of individual string values, and create a vocabulary from them.
- This vocabulary can have unlimited size or be capped, depending on the configuration options for this layer; if there are more unique values in the input than the maximum vocabulary size, the most frequent terms will be used to create the vocabulary.

In [ ]:
def lstm_preprocessing(text):

    text_vectorizer = TextVectorization(max_tokens=100000, # maxium number of words storing into vocab, larger the number of vocab larger the model would be.
                           output_sequence_length=None, # maxium length of the sentance in tokens. higher number would make larger model.
                           output_mode='int',#it will map every single word into integer values, taking words and converting them into integer.
                           standardize='lower_and_strip_punctuation', #to lower the text and strip out the punctuatoins.
                           split = 'whitespace'
                           )

    #learn all the words inside our training comment_text data. 
    text_vectorizer.adapt(text.values) 

    #vectorizing all comments 
    comment_vectorized = text_vectorizer(text.values)

    #returning text_vectorizer and vectorized commnets for reuse.
    return text_vectorizer, comment_vectorized

In [ ]:
if __name__== "__main__":
    text_vectorizer, comment_vectorized = lstm_preprocessing(train['comment_text'])

In [ ]:
train['comment_text'][0]

'explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired'

In [ ]:
#lets check how its converting comments into integer sequences.
text_vectorizer(train['comment_text'][0])

<tf.Tensor: shape=(45,), dtype=int64, numpy=
array([  651,    78,     2,   129,   134,   178,    32,   637,  4249,
       10741,   980,    88,   323,    50,  2116,    23, 10844,    55,
        6507,    16,    63,  2612,   151,     4,  2778,    38,   122,
        1137, 15517,  2643,     6,    48,    61,    23,   240,     2,
         354,    34,     2,    40,    30,   147,     4,    71,  3226])>

In [ ]:
#Creating tensorflow data pipeline 
BUFFER_SIZE = len(train)
BATCH_SIZE = 24
PREFETCH = tf.data.AUTOTUNE

def lstm_pipeline(vectorized_text,labels):
    dataset = tf.data.Dataset.from_tensor_slices((comment_vectorized, train_y))
    dataset = dataset.cache()
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(PREFETCH) 
    return dataset

In [ ]:
#splitting training, validation dataset
def data_split(dataset):
    train_dataset = dataset.take(int(len(dataset)*.8))  #taking 80% data as a training data
    val_dataset = dataset.skip(int(len(dataset)*.8)).take(int(len(dataset)*.2)) #skipping 80% data and taking remaining 20% dataset as validation dataset.
    return train_dataset,val_dataset

In [ ]:
if __name__== "__main__":
    dataset = lstm_pipeline(comment_vectorized,labels)

In [ ]:
if __name__== "__main__":
    train_dataset,val_dataset = data_split(dataset)

# Model Building

### Naive Bayes Baseline

### Logistic Regression 

In [ ]:
def logistic_regression(train_x,test_x):
    validation_score = []
    for label in labels:
        train_y = train[label]
        lr_model = LogisticRegression(solver='sag')
        lr_cv_score = np.mean(cross_val_score(lr_model, train_x, train_y, cv=3, scoring='roc_auc'))
        validation_score.append(lr_cv_score)
        lr_model.fit(train_x, train_y)
        #preds = lr_model.predict_proba(test_x)[:,-1]
        preds = lr_model.predict_proba(test_x)[:, 1]
        #submission[label] = preds
    print('Total CV score is {}'.format(np.mean(validation_score)))
    return lr_model,preds,validation_score

In [ ]:
if __name__== "__main__":
    lr_model,lr_preds,lr_score =logistic_regression(train_x,test_x)

Total CV score is 0.9848715601374182


### LSTM Bidirectional with Tokenization baseline model 

In [ ]:
def lstm_baseline(lstm_train_x,train_y):
    
    #creatign a sequential model.
    baseline_lstm = Sequential()
    
    baseline_model_inputs = Input(shape=(100, ))
    
     #since our max number of words are 100000, and input shape for the model is (100,None) since maxlen = 100.
    baseline_lstm = Embedding(100000+1, 128)(baseline_model_inputs)
    baseline_lstm = Bidirectional(LSTM(32))(baseline_lstm)
    baseline_lstm = Dropout(0.1)(baseline_lstm)
    baseline_lstm = Dense(128, activation="relu")(baseline_lstm)
    baseline_lstm = Dropout(0.1)(baseline_lstm)
    
     # using sigmoid to get values from 0,1 same as logistic regression. since there are six labels 
    baseline_model_outputs = Dense(6, activation="sigmoid")(baseline_lstm)
    lstm_baseline_model = Model(inputs=baseline_model_inputs, outputs=baseline_model_outputs)
    lstm_baseline_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    lstm_baseline_model.summary()
    
    return lstm_baseline_model

In [ ]:
if __name__== "__main__":
    lstm_baseline_model = lstm_baseline(lstm_train_x,train_y)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 128)          12800128  
                                                                 
 bidirectional (Bidirectiona  (None, 64)               41216     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                             

### LSTM Bidrectional TextVectorization model 

In [ ]:
def bidirectoinal_lstm(train_data,validation_data):
    
    lstm_model = Sequential()

    lstm_model.add(Embedding(1000001,32)) 

    forward_layer = LSTM(16,activation='tanh') #since we are using gpu in google colab that why activation is tanh else could have used relu.
    backward_layer = LSTM(16,go_backwards=True,activation='tanh')
    lstm_model.add(Bidirectional(forward_layer, backward_layer=backward_layer))

    lstm_model.add(Dense(128, activation='relu'))
    lstm_model.add(Dense(256, activation='relu'))
    lstm_model.add(Dense(128, activation='relu'))

    # using sigmoid to get values from 0,1 same as logistic regression. since there are six labels 
    lstm_model.add(Dense(6, activation='sigmoid'))

    #Compiling the model, since we want 6 values so using binarycrossentropy instead of catergory.
    lstm_model.compile(loss='BinaryCrossentropy', optimizer='Adam',metrics=['accuracy'])

    #Printing model summary
    lstm_model.summary()

    return lstm_model

In [ ]:
if __name__== "__main__":
    bidirectional_lstm = bidirectoinal_lstm(train_dataset,val_dataset)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          32000032  
                                                                 
 bidirectional_1 (Bidirectio  (None, 32)               6272      
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 128)               4224      
                                                                 
 dense_3 (Dense)             (None, 256)               33024     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 6)                 774       
                                                      

# Model Training

In [ ]:
def baseline(lstm_train_x,train_y):
  lstm_baseline_callback = EarlyStopping(monitor="val_loss", mode="min", patience=3)
  lstm_baseline_history = lstm_baseline_model.fit(lstm_train_x, train_y, batch_size=32, epochs=10, validation_split=0.1, callbacks=lstm_baseline_callback)
  baseline_lstm_saved = lstm_baseline_model.save('baseline_lstm_model.h5')
  plt.figure(figsize=(8,8))
  pd.DataFrame(lstm_baseline_history.history).plot()
  plt.show()
  return baseline_lstm_saved

In [ ]:
if __name__== "__main__":
  baseline_lstm_saved = baseline(lstm_train_x,train_y)

Epoch 1/10
4488/4488 [==============================] - 72s 14ms/step - loss: 0.1449 - accuracy: 0.9610 - val_loss: 0.1430 - val_accuracy: 0.9940
Epoch 2/10
4488/4488 [==============================] - 61s 14ms/step - loss: 0.1425 - accuracy: 0.9941 - val_loss: 0.1431 - val_accuracy: 0.9940
Epoch 3/10
4488/4488 [==============================] - 62s 14ms/step - loss: 0.1420 - accuracy: 0.9942 - val_loss: 0.1438 - val_accuracy: 0.9940
Epoch 4/10
4488/4488 [==============================] - 62s 14ms/step - loss: 0.1419 - accuracy: 0.9942 - val_loss: 0.1435 - val_accuracy: 0.9940


In [ ]:
def bi_lstm(train_dataset,val_dataset):
  bidirectional_lstm_history = bidirectional_lstm.fit(train_dataset, epochs=10,validation_data=val_dataset)
  bidirectonal_lstm_saved = bidirectional_lstm.save('lstm_bidirectional_model.h5')
  plt.figure(figsize=(8,8))
  pd.DataFrame(bidirectional_lstm_history.history).plot()
  plt.show()
  return bidirectonal_lstm_saved

In [ ]:
bidirectonal_lstm_saved = bi_lstm(train_dataset,val_dataset)

Epoch 1/10
5319/5319 [==============================] - 473s 88ms/step - loss: 0.0621 - accuracy: 0.9846 - val_loss: 0.0470 - val_accuracy: 0.9946
Epoch 2/10
5319/5319 [==============================] - 470s 88ms/step - loss: 0.0454 - accuracy: 0.9935 - val_loss: 0.0386 - val_accuracy: 0.9939
Epoch 3/10
5319/5319 [==============================] - 467s 88ms/step - loss: 0.0399 - accuracy: 0.9920 - val_loss: 0.0350 - val_accuracy: 0.9939
Epoch 4/10
5319/5319 [==============================] - 448s 84ms/step - loss: 0.0372 - accuracy: 0.9941 - val_loss: 0.0307 - val_accuracy: 0.9945
Epoch 5/10
5319/5319 [==============================] - 447s 84ms/step - loss: 0.0329 - accuracy: 0.9942 - val_loss: 0.0280 - val_accuracy: 0.9942
Epoch 6/10
5319/5319 [==============================] - 446s 84ms/step - loss: 0.0295 - accuracy: 0.9940 - val_loss: 0.0247 - val_accuracy: 0.9933
Epoch 7/10
5319/5319 [==============================] - 446s 84ms/step - loss: 0.0264 - accuracy: 0.9749 - val_loss: 0

# Perdictions 

In [ ]:
if __name__== "__main__":
    lr_model,lr_preds,lr_score =logistic_regression(train_x,test_x)

Total CV score is 0.9848710424515507


In [66]:
def baseline_pred():
  test['comment_text'] = test['comment_text'].apply(lambda x :clean_comments(x))
  baseline_test_x = test['comment_text'].values
  baseline_test_x = tokenizer.texts_to_sequences(baseline_test_x)
  baseline_test_x = keras.utils.pad_sequences(baseline_test_x, maxlen=100)
  baseline_pred = lstm_baseline_model.predict([baseline_test_x],batch_size = BATCH_SIZE)
  return baseline_test_x, baseline_pred 

if __name__ == "__main__":
    baseline_text_x,baseline_pred = baseline_pred()

6382/6382 [==============================] - 30s 5ms/step


In [73]:
def lstm_preds():
  test['comment_text'] = test['comment_text'].apply(lambda x :clean_comments(x))
  bidirectional_test_x= text_vectorizer(test['comment_text'])
  bidirectional_pred = bidirectional_lstm.predict((bidirectional_test_x),batch_size=24)
  bidirectional_pred = (bidirectional_pred > 0.5).astype(int)
  bidirectional_pred[np.isnan(bidirectional_pred)]=0
  return bidirectional_test_x,bidirectional_pred

if __name__ == "__main__":
    bidirectional_test_x,bidirectional_pred = lstm_preds()


6382/6382 [==============================] - 355s 56ms/step


# Model Evalution

In [75]:
#score for baseline lstm model
baseline_score = lstm_baseline_model.evaluate(baseline_text_x,baseline_pred, batch_size=32, verbose=2)

4787/4787 - 25s - loss: 0.1646 - accuracy: 1.0000 - 25s/epoch - 5ms/step


In [76]:
#score for bidirectional lstm with textvectorization
bidirectional_score = bidirectional_lstm.evaluate(bidirectional_test_x,bidirectional_pred, batch_size=32, verbose=2)

4787/4787 - 260s - loss: 0.0291 - accuracy: 0.9581 - 260s/epoch - 54ms/step


In [90]:
test.iloc[9873,1]

'this guy must not have seen torchwood where improbably and ridiculously everything is gay or bisexual big gay russell s big gay agenda is real'

In [88]:
baseline_pred[9873]

array([0.10973047, 0.01418762, 0.06372719, 0.01257128, 0.06312206,
       0.00413866], dtype=float32)

In [89]:
bidirectional_pred[9873]

array([1, 0, 0, 1, 0, 0])

In [81]:
labels

['toxic', 'severe_toxic', 'obscene', 'identity_hate', 'insult', 'threat']

In [91]:
def score_comment(comment):
    
    vectorized_comment = text_vectorizer([comment])
    results = bidirectional_lstm.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(train.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [93]:
score_comment('You sucks!!!')

1/1 [==============================] - 0s 21ms/step


'toxic: True\nsevere_toxic: False\nobscene: False\nthreat: False\ninsult: True\nidentity_hate: False\n'

In [95]:
score_comment('you are so stupid that you cant even speak!')

1/1 [==============================] - 0s 22ms/step


'toxic: True\nsevere_toxic: False\nobscene: False\nthreat: False\ninsult: True\nidentity_hate: False\n'

In [100]:
#creating a interface for Gradio app to evaluate the model.

interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [101]:
interface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://93f56788bf7dbd3f.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 24ms/step
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fd0472fa7d0>,
 'http://127.0.0.1:7860/',
 'https://93f56788bf7dbd3f.gradio.app')